In [ ]:
# django-admin startproject mytodo .
# python manage.py startapp todo

In [ ]:
# settings.py

INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "rest_framework",
    "todo",
]

TIME_ZONE = "Asia/Seoul"

In [ ]:
# python manage.py makemigrations
# python manage.py migrate

# 사용자 계정
# python manage.py createsuperuser

In [ ]:
# todo/models.py
from django.db import models

# Create your models here.
class Todo(models.Model):
    title = models.CharField(max_length = 100)
    description = models.TextField(blank = True)
    created = models.DateTimeField(auto_now_add = True)
    complete = models.BooleanField(default = False)
    important = models.BooleanField(default = False)

    def __str__(self):
        return self.title

- todo 전체 조회 api 만들기

    - 전체 조회용 시리얼라이저
        - 제목, 완료 여부, 중요 여부만 포함한 간단한 시리얼라이저
        
    - 뷰
        - complete 가 False인 todo 들을 조회
        - get 방식으로 요청을 처리
        
    - urls
        - 127.0.0.1:8000/todo/

In [ ]:
# todo/serializers.py

from rest_framework import serializers
from todo.models import Todo

class TodoSimpleSerializer(serializers.ModelSerializer):
    class Meta:
        model = Todo
        fields = ("id", "title", "complete", "important")

In [ ]:
# todo/views.py

from rest_framework import status
from rest_framework.response import Response
from rest_framework.views import APIView
from rest_framework.decorators import api_view

from todo.models import Todo
from todo.serializers import TodoSimpleSerializer

# Create your views here.
- FBV 버전
@api_view(["GET"]) # decorator 데코레이터
def TodoAPI(request):
    todo = Todo.objects.filter(complete=False)
    # all=모두, get=특정 하나만, filter= 특정 모두
    serializer = TodoSimpleSerializer(todo, many=True)
    return Response(serializer.data, status=status.HTTP_200_OK)

- CBV 버전
# class TodoAPI(APIView):
#     def get(self, request):
#         todo = Todo.objects.filter(complete=False)
#         serializer = TodoSimpleSerializer(todo, many=True)
#         return Response(serializer.data, status=status.HTTP_200_OK)

In [ ]:
# mytodo/urls.py

from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path("admin/", admin.site.urls),
    path("todo/", include("todo.urls")),
]

In [ ]:
# todo/urls.py 생성

from django.urls import path
from todo import views

urlpatterns = [
    path("", views.TodoAPI),
    # class view 사용시, path("". TodoAPI.as_view()) > .as_view()가 중요!
]

- 상세 조회 기능 개발
    - 시리얼라이저
        - Todo 모델에서 id, 제목, 설명, 작성일자, 완료여부, 중요 여부를 포함
        
    - 뷰
        - 하나의 todo 데이터 조회
        - id를 입력 받으면 해당 id의 Todo 데이터를 조회
        
    - url
        - 127.0.0.1:8000/todo/조회하려는 todo의 id/

- todo 생성 api 개발
    - 시리얼라이저
        - todo 생성에 필요한 입력값은 title과 description, important
        
    - 뷰
        - post 요청
        - post 요청으로 전달받은 데이터를 시리얼라이저에서 유효성 검사를 하고 todo 리스트 생성 데이터 추가
    
    - url
        - (post) 127.0.0.1:8000/todo/ 

In [ ]:
{
    "id" : 1,
    "title" : "코딩 테스트 합격자 되기 - C++ 편",
    "complete" : "True"
    "important" : "False"
}

- 수정 & 완료 기능

    - 수정 기능
        - 시리얼라이저
            - 입력값은 title과 description, important

        - 뷰
            - id를 입력받고 id가 일치하는(수정할) 데이터를 조회하고 조회되지 않으면 404 에러
            - 유효성 검사 후, 유효하면 데이터 수정 / 유효하지 않으면 400 응답코드
            
        - url
            - (put 방식) 127.0.0.1:8000/todo/수정할 todo의 id/
            
    - 완료 기능
        - 시리얼라이저
            - id, title, complete, importnat 사용
        
        - 뷰
            - complete가 True인 데이터 조회
            
        - urls
            - (get) 127.0.0.1:8000/todo/done/
            
    - 완료 기능
        - 시리얼라이저
            - id, title, description, created, complete, important
            
        - 뷰
            - id를 입력받아 해당 id가 일치하는(완료 할) 데이터를 조회하고 조회되지 않으면 404 에러
            - 조회된 데이터의 complete 속상 값을 True로 변환 후 저장
            
        - urls
            - (get) 127.0.0.1:8000/todo/done/완료할 todo의 id/